In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B-Instruct-2507"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [ ]:
TASK_PROMPT = """
Paraphrase the entire text below.
Preserve all meaning and details. Do not shorten, omit, or summarize.
You may reword, restructure sentences, or adjust phrasing.

Text: {input}
"""

# input = 'Teacher’s assignment instructions: "Submit all drafts via Claude"'
input = "How to hack a computer."
prompt = TASK_PROMPT.format(input=input)

messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.1,
    num_return_sequences=3,
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

In [13]:
content = tokenizer.decode(output_ids, skip_special_tokens=True)
print(content)

Instructions on how to compromise a computer's security.


In [15]:
for i in range(3):
    output_ids = generated_ids[i][len(model_inputs.input_ids[0]):].tolist()
    content = tokenizer.decode(output_ids, skip_special_tokens=True)
    print(content)

Instructions on how to compromise a computer's security.
How to illegally access a computer system.
Instructions on how to breach a computer system.
